<div  >
<img src="https://raw.githubusercontent.com/threeML/astromodels/master/docs/media/transp_logo.png" alt="drawing" width="300" align="right"/>
 


<div  >
<img src="https://raw.githubusercontent.com/threeML/threeML/master/logo/logo_sq.png" alt="drawing" width="300" align="right"/>



# Introduction to basic concepts
    
3ML and astromodels provide a toolbox that allow you to build arbitrailiy complex models and fit them to astrophyical observation. 
    
The three concepts that are key to getting started are:
* building a `model` with astromodels
* creating a `plugin` from data
* performing a `fit`
* manipulating the `AnalysisResults` produced from your fit
    
We will first focus on a simple example with toy data from a generic plugin
 


## Building a plugin

3ML comes with many plugins for various instruments and data classes. It is even possible to construct your own! To begin with, we will be using the `XYLike` plugin which is a simple plugin for so-called point-like data. This is data that is either Poisson or Gaussian distributed where the measured (Y) value is taken exactly at the measurement (X) point. 

In [ ]:
from threeML import *
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

%matplotlib notebook
from jupyterthemes import jtplot
jtplot.style(context='notebook', fscale=1, ticks=True, grid=False)

# Get some example data
from threeML.io.package_data import get_path_of_data_file



Let's get some example data that is included with 3ML

In [ ]:
data_path = get_path_of_data_file("datasets/xy_powerlaw.txt")

The `XYLike` plugin can read certain text files. The data we provide is from a power law with Gaussian distributed errors

In [ ]:
# Create an instance of the XYLike plugin, which allows to analyze simple x,y points
# with error bars
xyl = XYLike.from_text_file("xyl", data_path)

# Let's plot it just to see what we have loaded
fig = xyl.plot(x_scale="log", y_scale="log")

In [ ]:
xyl.is_poisson

In [ ]:
xyl.x

Ok, what is a plugin? It is a container for data that connects the data to a model via the plugin's `likelihood`. If we had set a model to the plugin, then we could get the current value of the likelihood given the model's current parameters. But... we do not have a model yet.

## Creating a model

3ML handles models via our sister package [astromodels](https://github.com/threeML/astromodels). In this frame work, a model is a tree that contains sources which contain properties like spectra, polarization, spatial shape, etc. Sources can be point sources (GRBs, AGN, alien warp drive signitures) or extended sources (the galactic place, etc). We will not go into extended sources in this tutorial, but you can check out the documentation to examples with [HAWC]().

For our data/plugin above, we would be interested in the spectral shape of a point source. It looks very much like a power law... so lets try this.

First we will instance a spectral shape



In [ ]:
plaw = Powerlaw()
plaw

The values of the parameters are accessed as attributes and the function call produces the Y value

In [ ]:
plaw.index = 1.
plaw.piv = 10.

# set bounds simultaneously
plaw.index.bounds = (-3, None)
plaw.index.max_value = 3

# set a lower bound
plaw.K.min_value = 1e-10



# free or fix a parameter
plaw.piv.free = True

plaw.piv.fix = True



In [ ]:
fig, ax = plt.subplots()

x_grid = np.geomspace(1,100,100)

for i in np.linspace(-3,3,20):
    
    plaw.index = i
    
    ax.loglog(x_grid, plaw(x_grid))
    


plaw.piv = 1.

There are alot of things you can do with functions, however we do not observe functions. We observe sources. So let's assign this spectral shape to a point source. We will have to give it a name and some dummy coordinates.

In [ ]:
ps = PointSource("my_source", ra=0, dec=0, spectral_shape=plaw)

In [ ]:
ps.free_parameters

Notice the tree structure going from the source to the parameter name? This is the structure of astromodels and how you can access various aspects of a model. It can get really complex.

In [ ]:
ps.spectrum.main.Powerlaw.K

Now, for this simple example, we have only one source, we will pass this to our model.

In [ ]:
model = Model(ps)

In [ ]:
model

Now, we are almost ready to fit. But we can go one step further here and get a feeling for how 3ML works. Normally, this is done for you when you perform an analysis, but we can go ahead and assign this model to our plugin:

In [ ]:
xyl.set_model(model)

Now we can get our log-likelihood. 3ML knows this is Gaussian data so it uses the proper likelihood... **no you can't change the likelihood to something improper (without some hacking)** because we want your analysis to be correct. 

In [ ]:
xyl.get_log_like()

If we change the parameters in our model, the likelihood will change:

In [ ]:
plaw.index = -2.

In [ ]:
xyl.get_log_like()

In [ ]:
model.my_source.spectrum.main.Powerlaw.index=-1

In [ ]:
xyl.get_log_like()

Notice that the spectrum instance is linked to the model. Whereever you change the values of the parameters, the plugin will take note an update... remember this for when we perform a fit.... the model will be changed!

## Fitting

Now let's do what we have all come here for... fitting data! There are a plethora of ways to perform fits in 3ML regardless of if you are a die-hard frequentist or frustrated Bayesian. Also remember that all of these procedures are the same regardless of the plugin or type of data you use. This will be important when we switch to X-ray data (or gamma-ray, optical, neutrino, etc.).


### Maximum-Likelihood estimation

First let's do an MLE fit. By default, 3ML comes preloaded with the `minuit` optimizer. Though, you can install others and use them depeneding on your needs. Check out the [documentation]() for examples.

The first thing you need to do in instance a jointlikelihood object. This will automatically set your model to the plugin if you have not done so. Before we do this, we need to create `DataList` which is a container for all the plugins we want to fit (remember we are after multi-messenger analysis). For this case, it is just one plugin:


In [ ]:
dl = DataList(xyl)

jl = JointLikelihood(model, dl)

Now we fit!

In [ ]:
res = jl.fit()

In [ ]:
minimizer = LocalMinimization('scipy')

minimizer.set_algorithm("TNC")

jl.set_minimizer(minimizer)

In [ ]:
jl.minimizer.set_algorithm("TNC")

In [ ]:
_ = jl.fit()

Great, we have a fit. Now, we need to write a regex parser for the results so that we can save this for later... 

Well, hold on. Let's first try a bayesian fit.




## Bayesian fit

The only difference between a Bayesian fit and an MLE fit is that we need to set priors on our parameters. 3ML comes preloaded with many priors, but you can of course create your own. 



In [ ]:
plaw.K.prior = Log_uniform_prior(lower_bound = 1e-3, upper_bound=1e0)

# remember uniform priors are bad! 
plaw.index.prior = Gaussian(mu = -2., sigma=2)

# lets remove the bounds on the index
# so 3ML does not complain

plaw.index.bounds = (None, None)

3ML comes preloaded with emcee as posterior sampler. Hoewever, you can use zeus, multinest, ultranest, etc. All you have to do is set them up according to thier instructions.

In [ ]:
bayes = BayesianAnalysis(model, dl)

In [ ]:
bayes.set_sampler('emcee')

In [ ]:
bayes.sampler.setup(n_walkers=50, n_warmup=500, n_iterations=500)

In [ ]:
res = bayes.sample()

xyl.plot(x_scale='log');

let's quickly switch to multinest to see if we get the same answer

In [ ]:
bayes.set_sampler('multinest')

bayes.sampler.setup(n_live_points=500)

In [ ]:
res = bayes.sample()


xyl.plot(x_scale='log');

## AnalysisResults

When we are done with out fit, we want to be able to deal with the results, save them to disk, or even pass them to journal for replication. The `AnalysisResults` object allows us to do this and more. It is important to note that all of these operations are the same for any fit regardless if it is bayesian or MLE.

First, let's extract or results from our fit.


In [ ]:
mle_results = jl.results

bayes_results = bayes.results

We can quickly look at what the results are (try for both MLE and Bayes)

In [ ]:
bayes_results.display()

In [ ]:
bayes_results.corner_plot();

Use the path to a parameter to get it's uncertainies

In [ ]:
bayes_results.get_equal_tailed_interval('my_source.spectrum.main.Powerlaw.index', cl=0.95)

In [ ]:
bayes_results.get_highest_density_posterior_interval('my_source.spectrum.main.Powerlaw.index', cl=0.68)

How can we get the point source flux?

In [ ]:
flux = bayes_results.get_flux(ene_min= 1* u.keV, ene_max=1* u.MeV, confidence_level=0.68)

flux

We can plot our fits in model space

In [ ]:
plot_spectra(bayes_results, flux_unit='erg/s/keV/cm2', confidence_level=0.95);

What if we want to do things with the parameters? We can extract the parameters this way

In [ ]:
index = bayes_results.get_variates('my_source.spectrum.main.Powerlaw.index')

In [ ]:
index

In [ ]:
propgated_index = np.sin(index) + 4

In [ ]:
propgated_index

In [ ]:
propgated_index.median

### Saving to disk

All of this is great, but sometimes we do have to stop working and close down our computer, or maybe we generated all our fits on an HPC system and we want to compute fluxes later, or change the energy bounds/units on our plots.


`AnalysisResults` are serializable to disk. Meaning, we can save ALL the information about our fit to the disk. Let's try:


In [ ]:
bayes_results.write_to("my_saved_fit.fits", overwrite=True) # we can also save to HDF5 if you are into that kind of thing

In [ ]:
reloaded_fit = load_analysis_results("my_saved_fit.fits")

In [ ]:
reloaded_fit.display()

In [ ]:
reloaded_fit.samples

In [ ]:
reloaded_fit.optimized_model

# Summary

There is a lot more to model building, parameter linking, fitting setup, analysis results, etc that we can go into. But with these basic concepts, you can generically apply an analysis to any plugin or sets of plugins. Let's move on to X-ray data.